In [31]:
!pip install ray

     |████████████████████████████████| 48.5MB 92kB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 1.3MB 41.2MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 204kB 58.3MB/s 
     |████████████████████████████████| 133kB 43.3MB/s 
     |████████████████████████████████| 2.9MB 40.5MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 143kB 46.2MB/s 
     |████████████████████████████████| 296kB 51.8MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=4d8b087fb4b651a27c95c1c5bef074f041c6043d4c40b676ca0fedf0681eb8da
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3

In [20]:
import numpy as np
import pandas as pd
import cProfile


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
        if(i==0):
          print("groupby_result:\r\n")
          print(groupby_result)
          print("result: \r\n")
          print(result)
    return result


def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result


def main():
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    print("data:\r\n")
    print(data)
    result_1 = target_mean_v1(data, 'y', 'x')
    result_2 = target_mean_v2(data, 'y', 'x')

    diff = np.linalg.norm(result_1 - result_2)
    print(diff)

cProfile.run('main()')

data:

      y  x
0     1  6
1     1  0
2     0  6
3     1  5
4     1  9
...  .. ..
4995  0  6
4996  1  5
4997  0  2
4998  1  8
4999  0  5

[5000 rows x 2 columns]
groupby_result:

          y      
       mean count
x                
0  0.481836   523
1  0.475191   524
2  0.485106   470
3  0.517717   508
4  0.485605   521
5  0.527668   506
6  0.460385   467
7  0.493697   476
8  0.510989   546
9  0.495633   458
result: 

[0.46038544 0.         0.         ... 0.         0.         0.        ]
0.0
         41385572 function calls (40935361 primitive calls) in 42.963 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    15002    0.020    0.000    0.161    0.000 <__array_function__ internals>:2(all)
     5000    0.010    0.000    0.552    0.000 <__array_function__ internals>:2(allclose)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amax)
        1    0.000    0.000    0.000    0.000 <__array_funct

In [ ]:
%load_ext Cython

In [37]:
%%cython

import numpy as np
cimport numpy as cnp
import pandas as pd
import cProfile
import ray


cdef target_mean_v1_c(data, y_name, x_name):
  cdef cnp.ndarray result = np.zeros(data.shape[0])
  cdef int n = data.shape[0]
  cdef int i
  for i in range(n):
    groupby_result=data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

cdef target_mean_v2_c(data, y_name, x_name):
    cdef cnp.ndarray result = np.zeros(data.shape[0])
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

cdef main_c():
  cdef cnp.ndarray y = np.random.randint(2, size=(5000, 1))
  cdef cnp.ndarray x = np.random.randint(10, size=(5000, 1))
  data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
  print("data:\r\n")
  print(data)
  cdef cnp.ndarray result_1 = target_mean_v1_c(data, 'y', 'x')
  cdef cnp.ndarray result_2 = target_mean_v2_c(data, 'y', 'x')
  print("result_1:\r\n")
  print(result_1)
  print("result_2:\r\n")
  print(result_2)
  diff = np.linalg.norm(result_1 - result_2)
  print("diff:\r\n")
  print(diff)


def main_p():
    ray.init()
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    result_1 = target_mean_v1_c(data, 'y', 'x')
    result_2 = target_mean_v2_c(data, 'y', 'x')

    diff = np.linalg.norm(result_1 - result_2)
    print(diff)

cProfile.run('main_p()')

0.0
         41360814 function calls (40910792 primitive calls) in 42.271 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    15000    0.019    0.000    0.156    0.000 <__array_function__ internals>:2(all)
     5000    0.011    0.000    0.547    0.000 <__array_function__ internals>:2(allclose)
    15000    0.025    0.000    0.287    0.000 <__array_function__ internals>:2(array_equal)
     5000    0.013    0.000    0.073    0.000 <__array_function__ internals>:2(bincount)
    15000    0.027    0.000    0.064    0.000 <__array_function__ internals>:2(can_cast)
     5001    0.012    0.000    0.051    0.000 <__array_function__ internals>:2(concatenate)
    10000    0.019    0.000    0.060    0.000 <__array_function__ internals>:2(copyto)
     5000    0.011    0.000    0.080    0.000 <__array_function__ internals>:2(cumsum)
     5000    0.015    0.000    0.188    0.000 <__array_function__ internals>:2(delete)
        1    0.000 